In [104]:
import pandas as pd
from decimal import *

In [105]:
csv_data = pd.read_csv(
    './assets/IHME-GBD_2017_DATA-37d305ef-1.csv',
    error_bad_lines=False
)
df = pd.DataFrame(csv_data)

In [106]:
del df['measure_id']
del df['measure_name']
df['location_id'] = df['location_id'].fillna(0)
df['location_name'] = df['location_name'].fillna('unknown')
df['sex_id'] = df['sex_id'].fillna(0)
df['sex_name'] = df['sex_name'].fillna('unknown')
del df['age_id']
del df['age_name']
del df['cause_id']
del df['cause_name']
del df['metric_id']
del df['metric_name']
df['year'] = df['year'].fillna(0)
df['val'] = df['val'].fillna(0)
df['upper'] = df['upper'].fillna(0)
df['lower'] = df['lower'].fillna(0)
df['sex_percentage'] = 1.0
df['rank'] = 0
df['percent_change'] = 0.0
df['average_percent_change'] = 0.0
df['raw_decrease_from_peak'] = 0.0
df['raw_increase_from_min'] = 0.0
df['avg_percent_change_since_peak'] = 0.0
df['avg_percent_change_since_min'] = 0.0
df['peak'] = 0.0
df['min'] = 0.0

In [107]:
# Let's find the val rank for each year/sex per country.
for year in df['year'].unique():
    for sex in df['sex_id'].unique():
        df_current = df[(df['sex_id']==sex) & (df['year']==year)].sort_values(by=['val'], ascending=False).reset_index()
        for index, row in df_current.iterrows():
            df.at[row['index'], 'rank'] = index + 1

In [108]:
# Calculate %change each year as well as data for calculating the sharpest increases/declines
for country in df['location_name'].unique():
    for sex in df['sex_id'].unique():
        df_country = df[(df['sex_id']==sex) & (df['location_name']==country)].sort_values(by=['year'])
        previous_val = df_country[(df_country['year']==1990)]['val']
        year_of_peak =  df_country.loc[df_country['val'].idxmax()]['year']
        year_of_min =  df_country.loc[df_country['val'].idxmin()]['year']
        total_change = 0
        total_change_since_peak = 0
        total_change_since_min = 0
        years_since_peak = 0
        years_since_min = 0
        for index, row in df_country[1:].iterrows():
            percent_change = (row['val'] - previous_val)/previous_val
            total_change += percent_change
            if(row['year'] > year_of_peak):
                total_change_since_peak += percent_change
                years_since_peak+=1
            if(row['year'] > year_of_min):
                total_change_since_min += percent_change
                years_since_min+=1
            previous_val = row['val']
            df.at[index, 'percent_change'] = percent_change
        df.at[index, 'average_percent_change'] = total_change/26
        try:
            df.at[index, 'avg_percent_change_since_peak'] = total_change_since_peak/(years_since_peak)
        except ZeroDivisionError:
            pass
        try:
            df.at[index, 'avg_percent_change_since_min'] = total_change_since_min/(years_since_min)
        except ZeroDivisionError:
            pass

        df.at[index, 'peak'] = Decimal(df_country['val'].max())
        df.at[index, 'min'] = Decimal(df_country['val'].min())
        df.at[index, 'raw_decrease_from_peak'] = float(df_country[(df['year']==2017)]['val']) - df_country['val'].max()
        df.at[index, 'raw_increase_from_min'] = float(df_country[(df['year']==2017)]['val']) - df_country['val'].min()

/Users/benjaminhurst/.local/share/virtualenvs/opioid_study-oZUPmX6j/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/benjaminhurst/.local/share/virtualenvs/opioid_study-oZUPmX6j/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [102]:
# The following block appends world data (annual means) to the dataset
df_world = pd.DataFrame(columns=['id', 'location_id', 'location_name', 'sex_id', 'sex_name', 'year', 'val', 'upper', 'lower', 'sex_percentage', 'rank', 'percent_change', 'average_percent_change', 'raw_decrease_from_peak', 'raw_increase_from_min', 'avg_percent_change_since_peak', 'avg_percent_change_since_min', 'peak', 'min'])

index = df.count()['location_id']

for year in df['year'].unique():
    for sex in df['sex_id'].unique():
        df_world.at[index, 'id'] = None
        df_world.at[index, 'location_id'] = int(0)
        df_world.at[index, 'location_name'] = 'World'
        df_world.at[index, 'sex_id'] = int(sex)
        df_world.at[index, 'sex_name'] = 'Male' if sex == 1 else 'Female' if sex == 2 else 'Both'
        df_world.at[index, 'year'] = int(year)
        df_world.at[index, 'val'] = df[(df['sex_id'] == sex) & (df['year'] == year)]['val'].mean()
        df_world.at[index, 'upper'] = df[(df['sex_id'] == sex) & (df['year'] == year)]['upper'].mean()
        df_world.at[index, 'lower'] = df[(df['sex_id'] == sex) & (df['year'] == year)]['lower'].mean()
        df_world.at[index, 'rank'] = int(0)
        df_world.at[index, 'sex_percentage'] = 1.0
        df_world.at[index, 'percent_change'] = 0.0
        df_world.at[index, 'average_percent_change'] = 0.0
        df_world.at[index, 'raw_decrease_from_peak'] = 0.0
        df_world.at[index, 'raw_increase_from_min'] = 0.0
        df_world.at[index, 'avg_percent_change_since_peak'] = 0.0
        df_world.at[index, 'avg_percent_change_since_min'] = 0.0
        df_world.at[index, 'peak'] = 0.0
        df_world.at[index, 'min'] = 0.0
        # df_world.at[index, 'id'] = index

        index += 1

df_world = df_world.sort_values(by=['year'])
# print(df_world)
for sex in df_world['sex_id'].unique():
    df_sex = df_world[(df_world['sex_id']==sex)].sort_values(by=['year'])
#     print(df_sex)
#     break
    previous_val = df_sex[(df_sex['year']==1990)]['val']
    total_percent_change = 0
    for index, row in df_sex[1:].iterrows():
#         break
        percent_change = float((row['val'] - previous_val)/previous_val)
        total_percent_change += percent_change
        previous_val = row['val']
        df_world.at[index, 'percent_change'] = percent_change
    df_world.at[index, 'average_percent_change'] = float(total_percent_change/26)

df = df.append(df_world, sort=True)

In [109]:
df_world

,id,location_id,location_name,sex_id,sex_name,year,val,upper,lower,sex_percentage,rank,percent_change,average_percent_change,raw_decrease_from_peak,raw_increase_from_min,avg_percent_change_since_peak,avg_percent_change_since_min,peak,min
16968,None,0,World,1,Male,1990,0.808026,1.01103,0.62223,1,0,0,0,0,0,0,0,0,0
16969,None,0,World,2,Female,1990,0.207061,0.256045,0.160143,1,0,0,0,0,0,0,0,0,0
16970,None,0,World,3,Both,1990,0.506177,0.615114,0.402634,1,0,0,0,0,0,0,0,0,0
16974,None,0,World,1,Male,1991,0.862912,1.07198,0.671625,1,0,0.0679268,0,0,0,0,0,0,0
16975,None,0,World,2,Female,1991,0.215708,0.265314,0.168327,1,0,0.0417605,0,0,0,0,0,0,0
16976,None,0,World,3,Both,1991,0.537849,0.649795,0.431973,1,0,0.0625715,0,0,0,0,0,0,0
16971,None,0,World,1,Male,1992,0.92377,1.13863,0.728825,1,0,0.0705257,0,0,0,0,0,0,0
16972,None,0,World,2,Female,1992,0.227739,0.278561,0.179519,1,0,0.0557781,0,0,0,0,0,0,0
16973,None,0,World,3,Both,1992,0.574216,0.689072,0.466528,1,0,0.0676164,0,0,0,0,0,0,0
16979,None,0,World,3,Both,1993,0.612418,0.730498,0.502131,1,0,0.0665283,0,0,0,0,0,0,0


In [110]:
# For each country determine what % of deaths are male/female

for country in df['location_name'].unique():
    for year in df['year'].unique():
        df_current = df[(df['location_name']==country) & (df['year']==year)]
        male_rate = float(df_current[(df['sex_id']==1)]['val'])
        female_rate = float(df_current[(df['sex_id']==2)]['val'])
        both_rate = float(df_current[(df['sex_id']==3)]['val'])
        female_percentage = (female_rate*(both_rate-male_rate))/(both_rate*(female_rate-male_rate))
        male_percentage = 1 - female_percentage
        df.at[df_current.index[0], 'sex_percentage'] = male_percentage
        df.at[df_current.index[0]+1, 'sex_percentage'] = female_percentage
        df.at[df_current.index[0]+1, 'sex_percentage'] = female_percentage

/Users/benjaminhurst/.local/share/virtualenvs/opioid_study-oZUPmX6j/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/benjaminhurst/.local/share/virtualenvs/opioid_study-oZUPmX6j/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/Users/benjaminhurst/.local/share/virtualenvs/opioid_study-oZUPmX6j/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [111]:
df

,location_id,location_name,sex_id,sex_name,year,val,upper,lower,sex_percentage,rank,percent_change,average_percent_change,raw_decrease_from_peak,raw_increase_from_min,avg_percent_change_since_peak,avg_percent_change_since_min,peak,min
0,108,Belize,1,Male,1990,0.111066,0.134317,0.090071,0.683366,186,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,108,Belize,2,Female,1990,0.052717,0.062200,0.043408,0.316634,160,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,108,Belize,3,Both,1990,0.082243,0.094777,0.069078,1.000000,186,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,7,North Korea,1,Male,1990,0.627025,0.912639,0.363522,0.689975,75,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,7,North Korea,2,Female,1990,0.249210,0.348083,0.160998,0.310025,53,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,7,North Korea,3,Both,1990,0.426544,0.579862,0.278203,1.000000,71,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,133,Venezuela,1,Male,1990,0.277605,0.326285,0.233146,0.701541,142,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,133,Venezuela,2,Female,1990,0.117293,0.133007,0.098561,0.298459,104,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,133,Venezuela,3,Both,1990,0.197173,0.222632,0.171501,1.000000,133,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,184,Mozambique,1,Male,1990,0.385147,0.612079,0.157109,0.881397,117,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
